In [1]:
import flwr as fl 
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import tasks as ts
import warnings
import argparse
import numpy as np




In [2]:
data_2 = pd.read_csv('./df_2.csv')

In [6]:
class CustomClient(fl.client.NumPyClient):

    def __init__(self):
        
        self.__public_key = self.load_key('public_key.pkl')
        self.__private_key = self.load_key('private_key.pkl')

    def load_key(self, file_path):
        
        with open(file_path, 'rb') as f:
            return pickle.load(f)

    def get_parameters(self, config):
       
        print('================== INITIAL PARAMS =====================')
        params = ts.get_model_parameters(model)
        print(params)
        return params

    def fit(self, parameters, config):
        
        ts.set_model_parameters(model, parameters)
        print('============================ PARAMETERS BEFORE LOCAL FIT===========================')
        print(parameters)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model.fit(X_train, y_train)
        
        print(f"Training finished for round {config['server_round']}")
        
        params_1 = ts.get_model_parameters(model)

        print(f'Clear: {params_1}')
        
        print('==================================== PARAMETERS AFTER FIT ===================== ')
        enc_result = self.encrypt_parameters(params_1)
        print(f'Encrypted: {enc_result}')
        
        return enc_result, len(X_train), {}

    def encrypt_parameters(self, parameters):
     
        encrypted_params = [(self.__public_key.encrypt(value),) for param in parameters for value in param.flatten()]
        enc_result = []
        index = 0

        for param in parameters:
            num_elements = param.size
            reshaped_array = np.array(encrypted_params[index:index + num_elements]).reshape(param.shape)
            enc_result.append(reshaped_array)
            index += num_elements

        return enc_result

    def evaluate(self, parameters, config):
        
        print('=========================== PARAMETERS AFTER AGG. BEFORE LOCAL EVAL =====================')
        print(parameters)
        dec_res = self.decrypt_parameters(parameters)
        print('========================== PARAMS BEFORE LOCAL EVAL =======================================')
        print(f'Decrypted parameters: {dec_res}')
        
        ts.set_model_parameters(model, dec_res)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        y_pred = model.predict(X_test)
        
        loss = log_loss(y_test, y_pred_proba)
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        f1 = f1_score(y_test, y_pred)
        
        return loss, len(X_test), {"accuracy": accuracy, "roc_auc": roc_auc, "f1-score": f1}

    def decrypt_parameters(self, parameters):
        
        decrypted_params = [(self.__private_key.decrypt(value),) for param in parameters for value in param.flatten()]
        dec_res = []
        index = 0

        for param in parameters:
            num_elements = param.size
            reshaped_array = np.array(decrypted_params[index:index + num_elements]).reshape(param.shape)
            dec_res.append(reshaped_array)
            index += num_elements

        return dec_res

if __name__ == "__main__":

    X, y = data_2.drop(columns=["Fraud"]), data_2["Fraud"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LogisticRegression(
        penalty='l2',
        class_weight={0: 1, 1: 10},
        solver='sag',
        max_iter=1
    )

 
    ts.set_initial_parameters(model)


    fl.client.start_client(
        server_address="127.0.0.1:8080",
        client=CustomClient()
    )


INFO :      
INFO :      Received: train message a88b005d-51d4-43dd-87b2-4a80d7173f00


============================ PARAMETERS BEFORE LOCAL FIT===========================
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([0.])]
Training finished for round 1
Clear: [array([[-1.23733611e-05, -6.45401946e-06, -6.10260467e-06,
        -3.84553461e-05, -1.99287934e-06, -6.45787855e-06,
        -2.83939393e-05, -1.06833419e-05, -1.65389310e-06,
        -3.76774125e-06, -1.84810243e-04, -2

INFO :      Sent reply


Encrypted: [array([[<phe.paillier.EncryptedNumber object at 0x000001C850F3D6C0>,
      dtype=object), array([<phe.paillier.EncryptedNumber object at 0x000001C850EEF610>],
      dtype=object)]


INFO :      
INFO :      Received: evaluate message a3847582-9875-436b-9d4f-11f3d470d900


...

INFO :      
INFO :      Received: evaluate message eeee9d3a-b195-436d-88bb-50f1e2852e7a


=========================== PARAMETERS AFTER AGG. BEFORE LOCAL EVAL =====================
[array([[<phe.paillier.EncryptedNumber object at 0x000001C85110F7C0>,
      dtype=object), array([<phe.paillier.EncryptedNumber object at 0x000001C8512E4400>],
      dtype=object)]


INFO :      Sent reply


========================== PARAMS BEFORE LOCAL EVAL =======================================
Decrypted parameters: [array([[-2.10123686e-05, -1.01521875e-05, -7.36104627e-06,
        -2.98661938e-05, -2.66445662e-06, -1.39505008e-05,
        -2.35604174e-05, -5.88624035e-06, -1.62202352e-06,
        -4.25692410e-06, -1.34363809e-04, -4.24973120e-06,
        -6.38924870e-06, -4.04033246e-06, -2.75034735e-06,
        -1.01908002e-03, -3.09473665e-05, -1.07385238e-03,
        -5.38723141e-06, -1.15720103e-05, -1.09640745e-05,
        -1.28637419e-05, -1.62449843e-05, -3.51865823e-07,
         1.33452435e-08, -1.48435432e-07, -9.81064193e-06,
         7.91218091e-07, -2.07860562e-06,  1.47881229e-07,
        -9.10998687e-06,  2.64590201e-06, -2.85018426e-06]]), array([-2.64590204e-06])]


INFO :      
INFO :      Received: reconnect message 330c4653-3781-4d1f-b33e-af1bc81c2630
INFO :      Disconnect and shut down
